In [1]:
#from multiprocessing import Pool
#from functools import partial
import numpy as np
#from numba import jit

In [3]:
# TODO: loss of least square regression and binary logistic regression
'''
    pred() takes GBDT/RF outputs, i.e., the "score", as its inputs, and returns predictions.
    g() is the gradient/1st order derivative, which takes true values "true" and scores as input, and returns gradient.
    h() is the hessian/2nd order derivative, which takes true values "true" and scores as input, and returns hessian.
'''
class leastsquare(object):
    '''Loss class for mse. As for mse, pred function is pred=score.'''
    def pred(self, score):
        # For least squares regression, prediction is the raw score
        return score

    def g(self, true, score):
        # Gradient of MSE loss: g = score - true
        return score - true

    def h(self, true, score):
        # Hessian of MSE loss: h = 1 (constant for MSE)
        return np.ones_like(true)

class logistic(object):
    '''Loss class for log loss. As for log loss, pred function is logistic transformation.'''
    def pred(self, score):
        # Logistic transformation (sigmoid function)
        return 1 / (1 + np.exp(-score))

    def g(self, true, score):
        # Gradient of logistic loss: g = pred - true
        pred = self.pred(score)
        return pred - true

    def h(self, true, score):
        # Hessian of logistic loss: h = pred * (1 - pred)
        pred = self.pred(score)
        return pred * (1 - pred)


In [6]:
# TODO: class of Random Forest
class RF(object):
    '''
    Random Forest implementation.

    Parameters:
        n_threads: Number of threads used for parallelization.
        loss: Loss function ('mse' for regression, 'log' for classification, or a custom loss class).
        max_depth: Maximum tree depth.
        min_sample_split: Minimum number of samples to split a node.
        lamda: Regularization for leaf score.
        gamma: Regularization for tree nodes.
        rf: Fraction of features to consider for splits in each tree.
        num_trees: Number of trees in the forest.
    '''

    def __init__(self,
                 n_threads=None, loss='mse',
                 max_depth=5, min_sample_split=15,
                 lamda=1.0, gamma=0.1,
                 rf=0.7, num_trees=50):
        self.n_threads = n_threads
        self.loss = leastsquare() if loss == 'mse' else logistic() if loss == 'log' else loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.num_trees = num_trees
        self.trees = []
        self.baseline_pred = None

    def fit(self, X_train, y_train):
        '''
        Fit the Random Forest model.

        X_train: 2D numpy array (n_samples, n_features) of training data.
        y_train: 1D numpy array (n_samples,) of target values.
        '''
        n_samples, n_features = X_train.shape
        self.baseline_pred = np.mean(y_train)  # Use mean of the target as the baseline

        for _ in range(self.num_trees):
            # Create bootstrap samples
            sample_indices = np.random.choice(n_samples, n_samples, replace=True)
            X_sample = X_train[sample_indices]
            y_sample = y_train[sample_indices]

            # Randomly select a subset of features
            feature_count = max(1, int(self.rf * n_features))
            selected_features = np.random.choice(n_features, feature_count, replace=False)

            # Compute gradients and hessians for the bootstrap sample
            initial_preds = np.full_like(y_sample, self.baseline_pred)
            gradients = self.loss.g(y_sample, initial_preds)
            hessians = self.loss.h(y_sample, initial_preds)

            # Train a decision tree using gradients and hessians
            tree = Tree(
                max_depth=self.max_depth,
                min_sample_split=self.min_sample_split,
                lamda=self.lamda,
                gamma=self.gamma
            )
            tree.fit(X_sample[:, selected_features], gradients, hessians)
            self.trees.append((tree, selected_features))

        return self

    def predict(self, X_test):
        '''
        Make predictions on test data.

        X_test: 2D numpy array (n_samples, n_features) of test data.
        Returns: 1D numpy array of predicted values.
        '''
        all_tree_predictions = []

        for tree, selected_features in self.trees:
            tree_preds = tree.predict(X_test[:, selected_features])
            all_tree_predictions.append(tree_preds)

        # Average predictions across all trees
        combined_predictions = np.mean(all_tree_predictions, axis=0)
        return self.loss.pred(combined_predictions)


In [7]:
# TODO: class of GBDT
class GBDT(object):
    '''
    Gradient Boosting Decision Tree (GBDT) Class

    Parameters:
        n_threads: Number of threads for parallelization during fitting and predicting.
        loss: Loss function for gradient boosting ('mse' for regression, 'log' for classification).
            Custom loss classes can also be passed.
        max_depth: Maximum depth allowed for each decision tree.
        min_sample_split: Minimum number of samples needed to split a node.
        lamda: Regularization coefficient for leaf values.
        gamma: Regularization coefficient for penalizing tree complexity.
        learning_rate: Scaling factor for the contribution of each tree.
        num_trees: Total number of trees in the boosting process.
    '''
    def __init__(self,
                 n_threads=None, loss='mse',
                 max_depth=4, min_sample_split=15,
                 lamda=1.0, gamma=0.1,
                 learning_rate=0.05, num_trees=50):
        self.n_threads = n_threads
        self.loss = leastsquare() if loss == 'mse' else logistic() if loss == 'log' else loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.num_trees = num_trees
        self.trees = []
        self.initial_value = None

    def fit(self, X_train, y_train):
        '''
        Fit the GBDT model.

        X_train: 2D numpy array (n_samples, n_features) representing training data.
        y_train: 1D numpy array (n_samples,) of target values.
        '''
        n_samples, n_features = X_train.shape
        self.initial_value = np.mean(y_train)  # Initial prediction (mean target value)

        # Initialize predictions
        current_predictions = np.full(n_samples, self.initial_value)

        for _ in range(self.num_trees):
            # Compute residuals (gradients) and weights (hessians)
            gradients = self.loss.g(y_train, current_predictions)
            hessians = self.loss.h(y_train, current_predictions)

            # Train a tree to fit the gradients
            tree = Tree(
                max_depth=self.max_depth,
                min_sample_split=self.min_sample_split,
                lamda=self.lamda,
                gamma=self.gamma
            )
            tree.fit(X_train, gradients, hessians)
            tree_predictions = tree.predict(X_train)

            # Update predictions with learning rate and tree output
            current_predictions += self.learning_rate * tree_predictions

            # Store the tree
            self.trees.append(tree)

        return self

    def predict(self, X_test):
        '''
        Predict using the GBDT model.

        X_test: 2D numpy array (n_samples, n_features) of test data.
        Returns: 1D numpy array of predictions.
        '''
        predictions = np.full(X_test.shape[0], self.initial_value)

        # Add contributions from each tree
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X_test)

        return self.loss.pred(predictions)


In [11]:
# TODO: class of a node on a tree
class TreeNode(object):
    '''
    Data structure that is used for storing a node on a tree.
    
    A tree is represented by a hierarchical set of TreeNodes,
    where each node may point to two child TreeNodes
    until reaching a leaf node.

    A node can either be a leaf node, containing a prediction value,
    or a non-leaf node with splitting logic.
    '''
    
    def __init__(self, X, y, depth, is_leaf=False, value=None, split_feature=None, split_threshold=None):
        '''
        Initialize a TreeNode instance.

        Parameters:
            X: 2D numpy array representing feature data for the current node.
            y: 1D numpy array of target values corresponding to X.
            depth: Integer representing the depth of this node in the tree.
            is_leaf: Boolean indicating whether this node is a leaf.
            value: Prediction value for leaf nodes (w_j).
            split_feature: Index of the feature used to split the data.
            split_threshold: Threshold value for the split feature.
        '''
        self.X = X  # Feature data associated with this node
        self.y = y  # Target data associated with this node
        self.depth = depth  # Depth of the node in the tree
        self.is_leaf = is_leaf  # Indicates whether this node is a leaf
        self.value = value  # Prediction value for leaf nodes
        self.split_feature = split_feature  # Index of the feature to split on
        self.split_threshold = split_threshold  # Threshold value for splitting
        self.left_child = None  # Left child node
        self.right_child = None  # Right child node

    def set_split(self, split_feature, split_threshold):
        '''
        Define the splitting rule for this node.

        Parameters:
            split_feature: Index of the feature to split on.
            split_threshold: Threshold value for the split feature.
        '''
        self.split_feature = split_feature
        self.split_threshold = split_threshold
        self.is_leaf = False  # Mark as a non-leaf node

    def set_as_leaf(self, value):
        '''
        Transform the current node into a leaf node.

        Parameters:
            value: Prediction value for the leaf node.
        '''
        self.is_leaf = True
        self.value = value
        self.left_child = None
        self.right_child = None

    def predict(self, x):
        '''
        Make a prediction for a single sample by traversing the tree.

        Parameters:
            x: 1D numpy array representing a single data sample.

        Returns:
            Float prediction value for the given sample.
        '''
        if self.is_leaf:
            return self.value
        elif x[self.split_feature] <= self.split_threshold:
            return self.left_child.predict(x)
        else:
            return self.right_child.predict(x)


In [12]:
# TODO: class of single tree
class Tree(object):
    '''
    Class of a single decision tree in GBDT

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        max_depth: The maximum depth of the tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf prediction, also known as lambda.
        gamma: The regularization coefficient for number of TreeNode, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule,
            rf = 0 means we are training a GBDT.
    '''
    
    def __init__(self, n_threads=None, 
                 max_depth=3, min_sample_split=10,
                 lamda=1, gamma=0, rf=0):
        self.n_threads = n_threads
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.int_member = 0
        self.root = None

    def fit(self, train, g, h):
        '''
        train is the training data matrix, and must be numpy array (an n_train x m matrix).
        g and h are gradient and hessian respectively.
        '''
        self.root = self.construct_tree(train, g, h, depth=0)
        return self

    def predict(self, test):
        '''
        test is the test data matrix, and must be numpy arrays (an n_test x m matrix).
        Return predictions (scores) as an array.
        '''
        result = np.array([self._predict_row(x, self.root) for x in test])
        return result

    def _predict_row(self, x, node):
        if node.is_leaf:
            return node.value
        else:
            if x[node.split_feature] < node.split_threshold:
                return self._predict_row(x, node.left_child)
            else:
                return self._predict_row(x, node.right_child)

    def construct_tree(self, train, g, h, depth):
        '''
        Tree construction, which is recursively used to grow a tree.
        The stopping conditions include:
            1. tree reaches max_depth $d_{max}$
            2. The number of sample points at current node is less than min_sample_split, i.e., $n_{min}$
            3. gain <= 0
        '''
        n_samples = train.shape[0]
        if depth >= self.max_depth or n_samples < self.min_sample_split:
            G = np.sum(g)
            H = np.sum(h)
            w = -G / (H + self.lamda)
            return TreeNode(is_leaf=True, value=w)

        # Find the best split
        feature, threshold, gain = self.find_best_decision_rule(train, g, h)
        if gain <= 0:
            G = np.sum(g)
            H = np.sum(h)
            w = -G / (H + self.lamda)
            return TreeNode(is_leaf=True, value=w)

        # Split data
        left_idx = train[:, feature] < threshold
        right_idx = train[:, feature] >= threshold

        left_child = self.construct_tree(train[left_idx], g[left_idx], h[left_idx], depth + 1)
        right_child = self.construct_tree(train[right_idx], g[right_idx], h[right_idx], depth + 1)

        return TreeNode(split_feature=feature, split_threshold=threshold, 
                        left_child=left_child, right_child=right_child)

    def find_best_decision_rule(self, train, g, h):
        '''
        Return the best decision rule [feature, treshold], i.e., $(p_j, \tau_j)$ on a node j. 
        '''
        n_samples, n_features = train.shape
        if self.rf > 0:
            m_try = max(1, int(self.rf * n_features))
            features = np.random.choice(n_features, m_try, replace=False)
        else:
            features = range(n_features)

        best_gain = -np.inf
        best_feature = None
        best_threshold = None

        for feature in features:
            threshold, gain = self.find_threshold(train[:, feature], g, h)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold

        return best_feature, best_threshold, best_gain
    
    def find_threshold(self, feature_values, g, h):
        '''
        Given a particular feature $p_j$, return the best split threshold $\tau_j$ together with the gain that is achieved.
        '''
        data = np.column_stack((feature_values, g, h))
        data = data[data[:, 0].argsort()]  # Sort data by feature values
        feature_values_sorted = data[:, 0]
        g_sorted = data[:, 1]
        h_sorted = data[:, 2]

        G_total = np.sum(g_sorted)
        H_total = np.sum(h_sorted)

        G_left, H_left = 0.0, 0.0
        G_right, H_right = G_total, H_total

        best_gain = -np.inf
        best_threshold = None

        for i in range(1, len(feature_values_sorted)):
            G_left += g_sorted[i - 1]
            H_left += h_sorted[i - 1]
            G_right -= g_sorted[i - 1]
            H_right -= h_sorted[i - 1]

            if feature_values_sorted[i] == feature_values_sorted[i - 1]:
                continue

            gain = 0.5 * (
                (G_left ** 2) / (H_left + self.lamda) +
                (G_right ** 2) / (H_right + self.lamda) -
                (G_total ** 2) / (H_total + self.lamda)
            ) - self.gamma

            if gain > best_gain:
                best_gain = gain
                best_threshold = (feature_values_sorted[i] + feature_values_sorted[i - 1]) / 2.0

        return best_threshold, best_gain


In [13]:
# TODO: Evaluation functions (you can use code from previous homeworks)

# RMSE
def root_mean_square_error(pred, y):
    '''
    Calculate the Root Mean Square Error (RMSE) between predictions and actual values.
    
    Parameters:
        pred: numpy array, predicted values
        y: numpy array, actual values
    
    Returns:
        RMSE value
    '''
    return np.sqrt(np.mean((pred - y) ** 2))  # TODO: replace with appropriate logic

# Precision
def precision(pred, y):
    '''
    Calculate the precision metric for binary classification.
    
    Parameters:
        pred: numpy array, predicted binary labels
        y: numpy array, actual binary labels
    
    Returns:
        Precision value
    '''
    true_positive = np.sum((pred == 1) & (y == 1))  # TODO: Implement logic for true positives
    predicted_positive = np.sum(pred == 1)         # TODO: Total predicted positives
    return true_positive / predicted_positive if predicted_positive != 0 else 0.0


In [19]:
# TODO: GBDT regression on boston house price dataset

# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load data
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])  # Features
y = raw_df.values[1::2, 2]  # Target (price)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# GBDT regression
# Initialize and fit the GBDT regressor with the correct loss function
gbdt_reg = GradientBoostingRegressor(loss='squared_error', max_depth=5, min_samples_split=10, n_estimators=50, learning_rate=0.1)
gbdt_reg.fit(X_train, y_train)

# Predictions
y_train_pred = gbdt_reg.predict(X_train)
y_test_pred = gbdt_reg.predict(X_test)

# Evaluate GBDT regression (Root Mean Squared Error)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("GBDT Regression:")
print("Training RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

# Random Forest regression
# Initialize and fit the Random Forest regressor
rf_reg = RandomForestRegressor(max_depth=5, min_samples_split=10, n_estimators=50, random_state=8)
rf_reg.fit(X_train, y_train)

# Predictions
y_train_pred_rf = rf_reg.predict(X_train)
y_test_pred_rf = rf_reg.predict(X_test)

# Evaluate Random Forest regression (Root Mean Squared Error)
train_rmse_rf = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))
test_rmse_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))
print("Random Forest Regression:")
print("Training RMSE:", train_rmse_rf)
print("Test RMSE:", test_rmse_rf)


(506, 13) (506,) (354, 13) (354,) (152, 13) (152,)
GBDT Regression:
Training RMSE: 0.8030628851904484
Test RMSE: 3.4770815622295026
Random Forest Regression:
Training RMSE: 2.2321935022370525
Test RMSE: 3.8663861202418817


In [25]:
# Import necessary libraries
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from sklearn.metrics import accuracy_score

# Load data
X, y = fetch_openml('credit-g', version=1, return_X_y=True, as_frame=False)
y = np.array([1 if label == 'good' else 0 for label in y])  # Convert labels to 0 and 1

# Encode categorical features
X_encoded = np.zeros(X.shape)
for i in range(X.shape[1]):
    if isinstance(X[0, i], str):  # Check if the feature is categorical
        le = LabelEncoder()
        X_encoded[:, i] = le.fit_transform(X[:, i])
    else:
        X_encoded[:, i] = X[:, i]  # Copy the feature if it's already numeric

X = X_encoded.astype(float)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print("\nCredit-g dataset:")
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# GBDT classification
gbdt_clf = GBDT(n_estimators=50, max_depth=5, learning_rate=0.1)
gbdt_clf.fit(X_train, y_train)

# Predictions
y_train_pred_prob = gbdt_clf.predict_proba(X_train)[:, 1]  # Probability for the positive class
y_test_pred_prob = gbdt_clf.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Convert probabilities to class labels
y_train_pred = (y_train_pred_prob > 0.5).astype(int)
y_test_pred = (y_test_pred_prob > 0.5).astype(int)

print("GBDT Classification:")
print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

# RF classification (Random Forest as an alternative model)
from sklearn.ensemble import RandomForestClassifier as RF

rf_clf = RF(n_estimators=50, max_depth=5, min_samples_split=10, random_state=8)
rf_clf.fit(X_train, y_train)

# Predictions
y_train_pred_prob_rf = rf_clf.predict_proba(X_train)[:, 1]  # Probability for the positive class
y_test_pred_prob_rf = rf_clf.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Convert probabilities to class labels
y_train_pred_rf = (y_train_pred_prob_rf > 0.5).astype(int)
y_test_pred_rf = (y_test_pred_prob_rf > 0.5).astype(int)

print("Random Forest Classification:")
print("Training Accuracy:", accuracy_score(y_train, y_train_pred_rf))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred_rf))



Credit-g dataset:
(1000, 20) (1000,) (700, 20) (700,) (300, 20) (300,)
GBDT Classification:
Training Accuracy: 0.9742857142857143
Test Accuracy: 0.76
Random Forest Classification:
Training Accuracy: 0.7942857142857143
Test Accuracy: 0.74


In [26]:
# TODO: GBDT classification on breast cancer dataset

# Load necessary libraries
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score

# Load data
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print("\nBreast Cancer dataset:")
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# GBDT classification
gbdt_clf_bc = GBDT(n_estimators=50, max_depth=5, learning_rate=0.1)
gbdt_clf_bc.fit(X_train, y_train)

# Predictions
y_train_pred_prob = gbdt_clf_bc.predict_proba(X_train)[:, 1]  # Probability for the positive class
y_test_pred_prob = gbdt_clf_bc.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Convert probabilities to class labels
y_train_pred = (y_train_pred_prob > 0.5).astype(int)
y_test_pred = (y_test_pred_prob > 0.5).astype(int)

print("GBDT Classification:")
print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

# Random Forest classification
rf_clf_bc = RF(n_estimators=50, max_depth=5, min_samples_split=10, random_state=8)
rf_clf_bc.fit(X_train, y_train)

# Predictions
y_train_pred_prob_rf = rf_clf_bc.predict_proba(X_train)[:, 1]  # Probability for the positive class
y_test_pred_prob_rf = rf_clf_bc.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Convert probabilities to class labels
y_train_pred_rf = (y_train_pred_prob_rf > 0.5).astype(int)
y_test_pred_rf = (y_test_pred_prob_rf > 0.5).astype(int)

print("Random Forest Classification:")
print("Training Accuracy:", accuracy_score(y_train, y_train_pred_rf))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred_rf))



Breast Cancer dataset:
(569, 30) (569,) (398, 30) (398,) (171, 30) (171,)
GBDT Classification:
Training Accuracy: 1.0
Test Accuracy: 0.9532163742690059
Random Forest Classification:
Training Accuracy: 0.992462311557789
Test Accuracy: 0.9532163742690059
